In [7]:
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers, models

In [8]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [9]:
## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

## Preprocessing input
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

In [10]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [11]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [12]:

from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 20s 112ms/step - loss: 1.8690 - accuracy: 0.5173 - val_loss: 1.0498 - val_accuracy: 0.5973
Epoch 2/50
65/65 [==============================] - 5s 83ms/step - loss: 0.7590 - accuracy: 0.7343 - val_loss: 0.9494 - val_accuracy: 0.6712
Epoch 3/50
65/65 [==============================] - 6s 86ms/step - loss: 0.4651 - accuracy: 0.8326 - val_loss: 0.9854 - val_accuracy: 0.6673
Epoch 4/50
65/65 [==============================] - 5s 84ms/step - loss: 0.2931 - accuracy: 0.8968 - val_loss: 1.0974 - val_accuracy: 0.6693
Epoch 5/50
65/65 [==============================] - 6s 88ms/step - loss: 0.1864 - accuracy: 0.9416 - val_loss: 1.0401 - val_accuracy: 0.6926
Epoch 6/50
65/65 [==============================] - 5s 84ms/step - loss: 0.1855 - accuracy: 0.9363 - val_loss: 1.1191 - val_accuracy: 0.6770
Epoch 7/50
65/65 [==============================] - 5s 84ms/step - loss: 0.1075 - accuracy: 0.9674 - val_loss: 1.2280 - val_accuracy: 0.6829
Epoch 8/50
